# The top 15 movies on IMDb

## Main objectives: 
    1. Fetch top 15 movies with a minimum of 100 votes
    2. Ranking = (numVotes / averageNumberOfVotes) * averageRating
    3. List of the title of these top 15 movies


### Method:
    1. Download the two .tsv files containing information on the titles and ratings from (https://datasets.imdbws.com/name.basics.tsv.gz) ... into the pyspark folder
    2. Import the files into Jupyter notebook
    3. Point to the files with abspath
    4. Read the .tsv file containing the rating informations into a dataframe and then convert it into rdd type
    5. Do same for the .tsv file containing the tile informations
    6. Filter all movies, short movies, tv episodes, etc. so that only those that have 100 or more number of votes remain
    7. Join data using the join function, using the 'tconst' as the key
    8. Filter the joined data so that only movies are left
    9. Calculate the ranking using ...
    10. Loop through the final list and print out top 15 movies

In [18]:
from pyspark import SparkContext

In [2]:
import pathlib
path_to_ratings = pathlib.Path('raw_data/title.ratings.tsv')
abs_path = os.path.abspath(path_to_ratings)
df_ratings = spark.read.csv(abs_path, sep=r'\t', header=True).select('tconst','averageRating','numVotes')
rdd_ratings = df_ratings.rdd.map(list)
rdd_ratings.top(3)

[['tt9916778', '7.3', '24'],
 ['tt9916766', '6.9', '14'],
 ['tt9916720', '6.0', '57']]

In [3]:
path_to_titles = pathlib.Path('raw_data/title.basics.tsv')
abs_path = os.path.abspath(path_to_titles)
df_titles = spark.read.csv(abs_path, sep=r'\t', header=True).select('tconst','titleType','primaryTitle')
rdd_titles = df_titles.rdd.map(list)
rdd_titles.top(3)

[['tt9916880', 'tvEpisode', 'Horrid Henry Knows It All'],
 ['tt9916856', 'short', 'The Wind'],
 ['tt9916852', 'tvEpisode', 'Episode #3.20']]

In [4]:
rdd_ratings = rdd_ratings.filter(lambda x: int(x[2])>=100)

In [5]:
rdd_ratings_pair = rdd_ratings.map(lambda r: (r[0], r))
rdd_titles_pair = rdd_titles.map(lambda r: (r[0], r))

In [6]:
rdd_joined = rdd_ratings_pair.join(rdd_titles_pair)
rdd_movies = rdd_joined.filter(lambda x: x[1][1][1]=='movie')
rdd_movies.take(1)

[('tt0000009',
  (['tt0000009', '5.9', '154'], ['tt0000009', 'movie', 'Miss Jerry']))]

In [7]:
rdd_movies_votes = rdd_movies.map(lambda x: int(x[1][0][2]))
average_no_votes = rdd_movies_votes.mean()
rdd_final = rdd_movies.sortBy(lambda x: (int(x[1][0][2])/average_no_votes)*float(x[1][0][1]), False).take(15)

In [8]:
top_15_list = []

for each_term in rdd_final:
    
    top_15_list.append(each_term[1][1][2])

In [9]:
top_15_list

['The Shawshank Redemption',
 'The Dark Knight',
 'Inception',
 'Fight Club',
 'Pulp Fiction',
 'Forrest Gump',
 'The Godfather',
 'The Lord of the Rings: The Return of the King',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Matrix',
 'The Lord of the Rings: The Two Towers',
 'The Dark Knight Rises',
 'Interstellar',
 'Se7en',
 'Gladiator']

In [17]:
for each_term in rdd_final:
    print(each_term[1][1][2])

The Shawshank Redemption
The Dark Knight
Inception
Fight Club
Pulp Fiction
Forrest Gump
The Godfather
The Lord of the Rings: The Return of the King
The Lord of the Rings: The Fellowship of the Ring
The Matrix
The Lord of the Rings: The Two Towers
The Dark Knight Rises
Interstellar
Se7en
Gladiator
